In [ ]:
spark.stop()

In [2]:
import os

AWS_S3_ENDPOINT = os.getenv("AWS_S3_ENDPOINT")
NESSIE_URI = os.getenv("NESSIE_URI")
WAREHOUSE = os.getenv("WAREHOUSE")
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")

In [3]:
import pyspark
from pyspark.sql import SparkSession

conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
        .set('spark.jars.packages', ",".join([
            'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.9.2',
            'org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.103.3',
            'org.apache.iceberg:iceberg-aws-bundle:1.9.2'
            ]))
        .set('spark.sql.extensions', ",".join([
            'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions',
            'org.projectnessie.spark.extensions.NessieSparkSessionExtensions'
        ]))
        .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.nessie.uri', NESSIE_URI)
        .set('spark.sql.catalog.nessie.ref', 'main')
        .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
        .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
        .set('spark.sql.catalog.nessie.s3.endpoint', AWS_S3_ENDPOINT)
        .set('spark.sql.catalog.nessie.warehouse', WAREHOUSE)
        .set('spark.sql.catalog.nessie.s3.path-style-access', 'true')
        .set('spark.sql.catalog.nessie.s3.access-key-id', AWS_ACCESS_KEY_ID)
        .set('spark.sql.catalog.nessie.s3.secret-access-key', AWS_SECRET_ACCESS_KEY)
        .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
)

# Start Spark Session
spark = SparkSession.builder.master("spark://spark-master:7077").config(conf=conf).getOrCreate()
print("Spark Running")

/usr/local/lib/python3.8/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found


:: loading settings :: url = jar:file:/usr/local/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12 added as a dependency
org.apache.iceberg#iceberg-aws-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6d057ce0-f597-4e11-867a-b9c5ee182b00;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.9.2 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12;0.103.3 in central
	found org.apache.iceberg#iceberg-aws-bundle;1.9.2 in central
downloading https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-spark-runtime-3.5_2.12/1.9.2/iceberg-spark-runtime-3.5_2.12-1.9.2.jar ...
	[SUCCESSFUL ] org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.9.2!iceberg-spark-runtime-3.5_2.12.jar (8149ms)
downloading https://repo1.maven.org/maven2/org/projectnessie

Spark Running


In [4]:
spark.sql("CREATE TABLE nessie.names (name STRING) USING iceberg;")
spark.sql("INSERT INTO nessie.names VALUES ('Alex Merced1'), ('Dipankar Mazumdar1'), ('Jason Hughes1')")
spark.sql("SELECT * FROM nessie.names;").show()

+------------------+
|              name|
+------------------+
|      Alex Merced1|
|Dipankar Mazumdar1|
|     Jason Hughes1|
+------------------+

